In [0]:
!nvidia-smi

Sun Sep 15 22:54:12 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 430.40       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
!curl ipecho.net/plain

35.204.53.0

In [0]:
!ls

sample_data


In [0]:
download_dataset =True
if download_dataset:
  !pip install kaggle --upgrade
  try:
    # You can access google drive documents at '/content/gdrive/My Drive'
    from google.colab import drive
    drive.mount('./gdrive')

    # Copy ssh key
    !mkdir -p ~/.kaggle
    !cp '/content/gdrive/My Drive/Colab Notebooks/kaggle.json' ~/.kaggle

  except ImportError:
      print("Google drive not available. You should add your kaggle key to ~/.kaggle")

  !mkdir -p kaggle/
  !cd kaggle; kaggle datasets download -d hellababorsa/pretain-aptos-253-190915;kaggle datasets download -d hellababorsa/mvcv42fivefold;kaggle competitions download -c aptos2019-blindness-detection
  !cd kaggle; unzip /content/kaggle/mvcv42fivefold.zip -d mycv42; unzip /content/kaggle/pretain-aptos-253-190915.zip -d pretain190915; unzip /content/kaggle/train_images.zip -d traindataset19
  data_directory = "kaggle/"

#### plan v174 efficientnetb5, lr2e-4, ep4, freeze all-1 till ep0, weights, optimized kappa, no weights, seed42, crop reflect circle, output layer with avg and maxpool and dropout0.3, RAdam wd 0.005 step 10 0.9, train2015valid2019 then finetune valid part of 2019 within same, autoaugment and cutout and rotate, mse+bce, 9th preprocess + 4th augment no randomcrop no clahe and replace dihedral with flipxx and add 1st solution parameter in jitter 

In [0]:
bsz=32
weight0 = 0.25#0.125#0.0276*4
weight1 = 0.25#0.5#0.0596*4
weight2 = 0.25#0.25#0.076*4
weight3 = 0.25#1.#0.0925*4
weight4 = 0.25#1.#0.1541*4
auxloss = 0.5
weight_decay = 0.005
num_epochs = 15#13#1#6
lr = 2e-4
n_freeze = 0
picSize=260+20
warmup = int(4*90000*0.05/32)
model_name = 'efficientnet-b2'
saved_pretrain_model = 'kaggle/pretain190915/model_ep4.bin'
output_unit = 1408 #2560#2048#int(3072/2)#1408
aux_loss = 0.5
loss_weights = (25810+2443+5292+873+708+39533+3762+7861+1214+1206)/(25810*weight0+2443*weight1+5292*weight2+873*weight3+708*weight4+39533*weight0+3762*weight1+7861*weight2+1214*weight3+1206*weight4)
#(57822*0.25+5662*0.25+12187*0.25+1964*0.25+1903*0.25)/(57822+5662+12187+1964+1903)
see_train = False
see_valid = False
train_before = True
loss_weights

4.0

In [0]:
!wget https://raw.githubusercontent.com/4uiiurz1/pytorch-auto-augment/master/auto_augment.py

--2019-09-15 22:58:47--  https://raw.githubusercontent.com/4uiiurz1/pytorch-auto-augment/master/auto_augment.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9703 (9.5K) [text/plain]
Saving to: ‘auto_augment.py’

auto_augment.py     100%[===================>]   9.48K  --.-KB/s    in 0s      

2019-09-15 22:58:47 (167 MB/s) - ‘auto_augment.py’ saved [9703/9703]



In [0]:
!pip install --upgrade efficientnet-pytorch

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.4.0-cp36-none-any.whl size=11149 sha256=24598614d6dad5c54db26c58a5072c55c86b934ebcd51f55100e1b1a6cdd4a08
  Stored in directory: /root/.cache/pip/wheels/27/56/13/5bdaa98ca8bd7d5da65cc741987dd14391b87fa1a09081d17a
Successfully built efficientnet-pytorch


In [0]:
! git clone https://github.com/NVIDIA/apex
!pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Cloning into 'apex'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 5433 (delta 1), reused 3 (delta 1), pack-reused 5422
Receiving objects: 100% (5433/5433), 13.33 MiB | 5.28 MiB/s, done.
Resolving deltas: 100% (3512/3512), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:243: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-r15re3bp
Created temporary directory: /tmp/pip-req-tracker-pjs4rw11
Created requirements tracker '/tmp/pip-req-tracker-pjs4rw11'
Created temporary directory: /tmp/pip-install-sdrtvp5n
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-c9h2m6um
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-pjs4rw11'
    Running setup.py (path:/tm

In [0]:
!rm -r apex/

In [0]:
from auto_augment import AutoAugment, Cutout
from sklearn.metrics import cohen_kappa_score
import numpy as np
import pandas as pd
import numpy as np
from apex import amp
import time
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm
from albumentations import Compose, RandomBrightnessContrast, ShiftScaleRotate,augmentations
from albumentations.pytorch import ToTensor
import albumentations as A
import random
from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import torch.nn.functional as F
import os
import cv2
import sys
#package_dir = "../input/pretrained-models/pretrained-models/pretrained-models.pytorch-master/"
#sys.path.insert(0, package_dir)
#import pretrainedmodels
device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True
from efficientnet_pytorch import EfficientNet

In [0]:
def seed_everything(SEED):
    random.seed(SEED)
    os.environ['PYTHONHASHSEED'] = str(SEED)
    np.random.seed(SEED)
    torch.manual_seed(SEED)
    torch.cuda.manual_seed(SEED)
    torch.backends.cudnn.deterministic = True

seed_everything(42)

In [0]:
import numpy as np
import pandas as pd
import os
import scipy as sp
from functools import partial
from sklearn import metrics
from collections import Counter
import json
class OptimizedRounder(object):
    def __init__(self):
        self.coef_ = 0

    def _kappa_loss(self, coef, X, y):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4

        ll = metrics.cohen_kappa_score(y, X_p, weights='quadratic')
        return -ll

    def fit(self, X, y):
        loss_partial = partial(self._kappa_loss, X=X, y=y)
        initial_coef = [0.5, 1.5, 2.5, 3.5]
        self.coef_ = sp.optimize.minimize(loss_partial, initial_coef, method='nelder-mead')

    def predict(self, X, coef):
        X_p = np.copy(X)
        for i, pred in enumerate(X_p):
            if pred < coef[0]:
                X_p[i] = 0
            elif pred >= coef[0] and pred < coef[1]:
                X_p[i] = 1
            elif pred >= coef[1] and pred < coef[2]:
                X_p[i] = 2
            elif pred >= coef[2] and pred < coef[3]:
                X_p[i] = 3
            else:
                X_p[i] = 4
        return X_p

    def coefficients(self):
        return self.coef_['x']

In [0]:
def crop_image_from_gray(img, tol=7):
    """
    Crop out black borders
    https://www.kaggle.com/ratthachat/aptos-updated-preprocessing-ben-s-cropping
    
    NOTE: This was used to generate the pre-processed dataset
    """

    if img.ndim == 2:
        mask = img > tol
        return img[np.ix_(mask.any(1), mask.any(0))]
    elif img.ndim == 3:
        gray_img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        mask = gray_img > tol
        check_shape = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))].shape[0]
        if (check_shape == 0):
            return img
        else:
            img1 = img[:, :, 0][np.ix_(mask.any(1), mask.any(0))]
            img2 = img[:, :, 1][np.ix_(mask.any(1), mask.any(0))]
            img3 = img[:, :, 2][np.ix_(mask.any(1), mask.any(0))]
            img = np.stack([img1, img2, img3], axis=-1)
        return img


def circle_crop(img):
    """
    Create circular crop around image centre
    
    NOTE: This was used to generate the pre-processed dataset
    """

    #img = crop_image_from_gray(img)

    height, width, depth = img.shape
    largest_side = np.max((height, width))
    img = cv2.resize(img, (largest_side, largest_side))

    height, width, depth = img.shape

    x = int(width / 2)
    y = int(height / 2)
    r = np.amin((x, y))

    circle_img = np.zeros((height, width), np.uint8)
    cv2.circle(circle_img, (x, y), int(r), 1, thickness=-1)
    img = cv2.bitwise_and(img, img, mask=circle_img)
    img = crop_image_from_gray(img)

    return img

def make_square(img, fill_color=(0, 0, 0, 0)):              #def make_square(im, fill_color=(0, 0, 0, 0)):
    if img.shape[0] != img.shape[1]:
        s = max(img.shape[0:2])
        #Creating a dark square with NUMPY  
        f = np.zeros((s,s,3),np.uint8)

        #Getting the centering position
        ax,ay = (s - img.shape[1])//2,(s - img.shape[0])//2

        #Pasting the 'image' in a centering position
        f[ay:img.shape[0]+ay,ax:ax+img.shape[1]] = img
        return f
    else:
        return img

# Optimizer define

In [0]:
import math
import torch
from torch.optim.optimizer import Optimizer, required

class RAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(RAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(RAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]
                if state['step'] == buffered[0]:
                    N_sma, step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        step_size = group['lr'] / (1 - beta1 ** state['step'])
                    buffered[2] = step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:            
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

class PlainRAdam(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)

        super(PlainRAdam, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(PlainRAdam, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('RAdam does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                state['step'] += 1
                beta2_t = beta2 ** state['step']
                N_sma_max = 2 / (1 - beta2) - 1
                N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                if N_sma >= 5:                    
                    step_size = group['lr'] * math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    p_data_fp32.addcdiv_(-step_size, exp_avg, denom)
                else:
                    step_size = group['lr'] / (1 - beta1 ** state['step'])
                    p_data_fp32.add_(-step_size, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss


class AdamW(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0, warmup = 0):
        defaults = dict(lr=lr, betas=betas, eps=eps,
                        weight_decay=weight_decay, warmup = warmup)
        super(AdamW, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(AdamW, self).__setstate__(state)

    def step(self, closure=None):
        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                state['step'] += 1

                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)
                exp_avg.mul_(beta1).add_(1 - beta1, grad)

                denom = exp_avg_sq.sqrt().add_(group['eps'])
                bias_correction1 = 1 - beta1 ** state['step']
                bias_correction2 = 1 - beta2 ** state['step']
                
                if group['warmup'] > state['step']:
                    scheduled_lr = 1e-8 + state['step'] * group['lr'] / group['warmup']
                else:
                    scheduled_lr = group['lr']

                step_size = group['lr'] * math.sqrt(bias_correction2) / bias_correction1
                
                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * scheduled_lr, p_data_fp32)

                p_data_fp32.addcdiv_(-step_size, exp_avg, denom)

                p.data.copy_(p_data_fp32)

        return loss

# Dataset Class

In [0]:
class RetinopathyDatasetTrain3(Dataset):

    def __init__(self, csv_file,transform):
        self.transform = transform
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #img_name = os.path.join('../input/resized-2015-2019-blindness-detection-images/resized train 19', self.data.loc[idx, 'id_code'] + '.jpg')
        #img_name = os.path.join('../input/cv42croppadcenter260/train/kaggle/working/', self.data.loc[idx, 'id_code'] + '.jpeg')
        img_name = os.path.join('kaggle/traindataset19', self.data.loc[idx, 'id_code'] + '.png')
        #image = Image.open(img_name)
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
        image = make_square(image)
        #image = image[:,:,::-1]
        #image = circle_crop(image)
        image = Image.fromarray(image)
        label = torch.tensor(self.data.loc[idx, 'diagnosis'])
        image = self.transform(image)
        if label == torch.tensor(0):
            weight = weight0
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],0,0,0,0])
        elif label == torch.tensor(1):
            weight = weight1
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],1,0,0,0])
        elif label == torch.tensor(2):
            weight = weight2
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],1,1,0,0])
        elif label == torch.tensor(3):
            weight = weight3
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],1,1,1,0])
        else:
            weight = weight4
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],1,1,1,1])
        return {'image': image,#transforms.ToTensor()(image),#image,
                'labels': label,
                'weight': weight
                }

In [0]:
class RetinopathyDatasetValid3(Dataset):

    def __init__(self, csv_file,transform):
        self.transform = transform
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        #img_name = os.path.join('../input/resized-2015-2019-blindness-detection-images/resized train 19', self.data.loc[idx, 'id_code'] + '.jpg')
        #img_name = os.path.join('../input/cv42croppadcenter260/valid/kaggle/working/', self.data.loc[idx, 'id_code'] + '.jpeg')
        img_name = os.path.join('kaggle/traindataset19', self.data.loc[idx, 'id_code'] + '.png')
        #image = Image.open(img_name)
        image = cv2.imread(img_name)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = crop_image_from_gray(image)
        image = make_square(image)
        #image = image[:,:,::-1]
        #image = circle_crop(image)
        image = Image.fromarray(image)
        label = torch.tensor(self.data.loc[idx, 'diagnosis'])
        image = self.transform(image)
        if label == torch.tensor(0):
            weight = weight0
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],0,0,0,0])
        elif label == torch.tensor(1):
            weight = weight1
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],1,0,0,0])
        elif label == torch.tensor(2):
            weight = weight2
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],1,1,0,0])
        elif label == torch.tensor(3):
            weight = weight3
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],1,1,1,0])
        else:
            weight = weight4
            label = torch.tensor([self.data.loc[idx, 'diagnosis'],1,1,1,1])
        return {'image': image,#transforms.ToTensor()(image),#image,
                'labels': label,
                'weight': weight
                }

# Get the model

In [0]:
model_raw = EfficientNet.from_name(model_name) 
#torchvision.models.resnext101_32x8d(pretrained=True)#torchvision.models.resnet101(pretrained=False) #
#model.load_state_dict(torch.load("../input/pytorch-pretrained-models/resnet101-5d3b4d8f.pth"))
#model.avg_pool = nn.AdaptiveAvgPool2d(1)
#num_features = model.fc.in_features
#modules=list(model_raw.children())[:-1]
resnext101=model_raw#nn.Sequential(*modules)
#for p in model_raw.parameters():
#    p.requires_grad = False

#model.fc = nn.Linear(2048, 1)
#model = pretrainedmodels.__dict__['resnet101'](pretrained='imagenet')
'''model.avg_pool = nn.AdaptiveAvgPool2d(1)
model.last_linear = nn.Sequential(
                          nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                          nn.Dropout(p=0.25),
                          nn.Linear(in_features=2048, out_features=2048, bias=True),
                          nn.ReLU(),
                          nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
                          nn.Dropout(p=0.5),
                          nn.Linear(in_features=2048, out_features=1, bias=True),
                         )'''
#num_features = model.fc.in_features
#model.fc = nn.Linear(2048, 1)
#model.load_state_dict(torch.load("../input/pytorch-pretrained-models/resnet101-5d3b4d8f.pth"))
#model = model.to(device)

'model.avg_pool = nn.AdaptiveAvgPool2d(1)\nmodel.last_linear = nn.Sequential(\n                          nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),\n                          nn.Dropout(p=0.25),\n                          nn.Linear(in_features=2048, out_features=2048, bias=True),\n                          nn.ReLU(),\n                          nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),\n                          nn.Dropout(p=0.5),\n                          nn.Linear(in_features=2048, out_features=1, bias=True),\n                         )'

In [0]:
class NeuralNet(nn.Module):
    def __init__(self,resnext101):
        super(NeuralNet, self).__init__()
        self.model = resnext101
        self.avg_layer = nn.AdaptiveAvgPool2d(output_size=(1, 1))
        self.max_layer = nn.AdaptiveMaxPool2d(output_size=(1, 1))
        self.dropout = nn.Dropout(0.3)
        self.linear = nn.Linear(output_unit*2, 1+4)#2048#2560
        #self.linear2 = nn.Linear(2048, 1+4)

    def forward(self, x):
        model_out = self.model.extract_features(x)#self.model(x)
        avg_pool = torch.squeeze(self.avg_layer(model_out))
        max_pool = torch.squeeze(self.max_layer(model_out))
        total_pool = torch.cat([avg_pool,max_pool],dim=-1)
        dropout_out = self.dropout(total_pool)
        logits = self.linear(dropout_out)
        #out = torch.sigmoid(logits)
        #output = self.linear2(F.relu(logits)+logits)
        #logits = self.linear(torch.cat([avg_pool,max_pool]))
        return logits

In [0]:
model = NeuralNet(resnext101)
model = model.to(device)

In [0]:
#!pip install torchsummary
#from torchsummary import summary
#model_raw = model_raw.to(device)
#model_raw(torch.cuda.HalfTensor(np.random.normal(0,1,(4,3, 256, 256))))
#summary(model, (3, 456, 456))
#model_raw.extract_features(torch.cuda.HalfTensor(np.random.normal(0,1,(4,3, 256, 256)))).shape

# Create dataset + optimizer

In [0]:
train_transform = transforms.Compose([
    transforms.Resize((picSize, picSize)),
    transforms.CenterCrop(picSize-20),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation((0, 360)),
    transforms.ColorJitter(contrast=(0.9, 1.1)),
    Cutout(),
    transforms.RandomPerspective(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])
test_transform = transforms.Compose([
    transforms.Resize((picSize, picSize)),
    #transforms.ColorJitter(brightness=20., contrast=(0.9, 1.1),saturation=20.),
    #transforms.RandomPerspective(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Training Loop

In [0]:
torch.cuda.empty_cache()

In [0]:
#%%time
for iiii in range(5):
    model.load_state_dict(torch.load(saved_pretrain_model))
    if iiii==0:
        train_dataset3 = RetinopathyDatasetTrain3(csv_file='kaggle/mycv42/cv42trainzero.csv',
                                               transform = train_transform)#test_transform
        valid_dataset3 = RetinopathyDatasetValid3(csv_file='kaggle/mycv42/cv42validzero.csv',
                                               transform = test_transform)
    if iiii==1:
        train_dataset3 = RetinopathyDatasetTrain3(csv_file='kaggle/mycv42/cv42trainone.csv',
                                               transform = train_transform)#test_transform
        valid_dataset3 = RetinopathyDatasetValid3(csv_file='kaggle/mycv42/cv42validone.csv',
                                               transform = test_transform)
    if iiii==2:
        train_dataset3 = RetinopathyDatasetTrain3(csv_file='kaggle/mycv42/cv42traintwo.csv',
                                               transform = train_transform)#test_transform
        valid_dataset3 = RetinopathyDatasetValid3(csv_file='kaggle/mycv42/cv42validtwo.csv',
                                               transform = test_transform)
    if iiii==3:
        train_dataset3 = RetinopathyDatasetTrain3(csv_file='kaggle/mycv42/cv42trainthree.csv',
                                               transform = train_transform)#test_transform
        valid_dataset3 = RetinopathyDatasetValid3(csv_file='kaggle/mycv42/cv42validthree.csv',
                                               transform = test_transform)
    if iiii==4:
        train_dataset3 = RetinopathyDatasetTrain3(csv_file='kaggle/mycv42/cv42trainfour.csv',
                                               transform = train_transform)#test_transform
        valid_dataset3 = RetinopathyDatasetValid3(csv_file='kaggle/mycv42/cv42validfour.csv',
                                               transform = test_transform)
    

    train_dataset = train_dataset3#torch.utils.data.ConcatDataset([train_dataset1,train_dataset2,valid_dataset1,valid_dataset2])
    valid_dataset = valid_dataset3#torch.utils.data.ConcatDataset([train_dataset3,valid_dataset3])
    valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=bsz, shuffle=False, num_workers=4)
    print(len(train_dataset),len(valid_dataset))
    if train_before == True:
        accumulation_steps = int(32/bsz)
        print(loss_weights)
        '''plist = [
                 {'params': model.layer4.parameters(), 'lr': 1e-4, 'weight': 0.001},
                 {'params': model.last_linear.parameters(), 'lr': 1e-3}
                 ]'''
        #optimizer = optim.Adam(model.parameters(),lr=2e-4) #optim.Adam(plist, lr=2.5e-4)
        optimizer = RAdam(model.parameters(),lr=lr,weight_decay=weight_decay)#RAdam(model.parameters(),lr=2e-4) #AdamW(model.parameters(),lr=2e-4,warmup = warmup)
        model, optimizer = amp.initialize(model, optimizer,opt_level="O1")
        scheduler = lr_scheduler.CosineAnnealingLR(optimizer, int((num_epochs*len(train_dataset))))#+new_num_epochs*2*(len(train_dataset3)+len(valid_dataset3)))/32))#StepLR(optimizer, step_size=10,gamma=0.9999)
        #scheduler = lr_scheduler.MultiStepLR(optimizer, milestones=[10, 16, 22], gamma=0.1)
        #scheduler = lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda_rule)
        since = time.time()
        #criterion = nn.MSELoss()
        def criterion(weights,outputs,labels):
            #loss = torch.mean(loss_weights*weights*nn.MSELoss(reduction='none')(outputs,labels))
            #loss = loss_weights*torch.mean(weights*nn.CrossEntropyLoss(reduction='none')(outputs,labels.long()))
            #loss = loss_weights*nn.BCEWithLogitsLoss(weight=weights)(outputs,labels)
            loss = torch.mean(loss_weights*weights*nn.MSELoss(reduction='none')(outputs[:,0],labels[:,0]))+auxloss*nn.BCEWithLogitsLoss()(outputs[:,1:],labels[:,1:])
            return loss
            #torch.mean(loss_weights*weights*nn.MSELoss(reduction='none')(outputs,labels))


        for epoch in range(num_epochs):
            if epoch == n_freeze:                
                for param in model.parameters():
                    param.requires_grad = True 
            #if epoch == 9:
            #    for g in optimizer.param_groups:
            #        g['lr'] = 1e-4
            #elif epoch == 15:
            #    for g in optimizer.param_groups:
            #        g['lr'] = 1e-5   
            #elif epoch == 21:
            #    for g in optimizer.param_groups:
            #        g['lr'] = 1e-6      
            data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=bsz, shuffle=True, num_workers=4)
            print('fold {} Epoch {}/{}'.format(iiii, epoch, num_epochs - 1))
            print('-' * 10)
            model.train()
            running_loss = 0.0
            if see_train == True:
                tk0 = tqdm(data_loader, total=int(len(data_loader)))#tk0 = tqdm(data_loader, total=int(len(data_loader)))#tk0 = data_loader
            else:
                tk0 = data_loader
            counter = 0
            optimizer.zero_grad()
            for bi, d in enumerate(tk0):
                #if bi>=10:
                #    break
                inputs = d["image"]
                labels = d["labels"].view(-1, 1+4)
                weights = d["weight"].view(-1, 1)
                #print(weights)
                inputs = inputs.to(device, dtype=torch.float)
                labels = labels.to(device, dtype=torch.float)
                weights = weights.to(device, dtype=torch.float)
                #optimizer.zero_grad()
                with torch.set_grad_enabled(True):
                    outputs = model(inputs)
                    loss = criterion(weights,outputs, labels)
                    with amp.scale_loss(loss, optimizer) as scaled_loss:
                        scaled_loss.backward()
                    if (bi+1) % accumulation_steps == 0:             # Wait for several backward steps
                        optimizer.step()                            # Now we can do an optimizer step
                        scheduler.step()
                        optimizer.zero_grad()
                    #optimizer.step()
                running_loss += loss.item() * inputs.size(0)
                counter += 1
                #tk0.set_postfix(loss=(running_loss / (counter * data_loader.batch_size)))
            epoch_loss = running_loss / len(data_loader)

            ##validation loss
            if see_valid == True:
                tk0 = tqdm(valid_data_loader, total=int(len(valid_data_loader)))#valid_data_loader
            else:
                tk0 = valid_data_loader#tqdm(valid_data_loader, total=int(len(valid_data_loader)))#valid_data_loader
            model.eval()
            valid_preds = np.array([])#np.zeros((len(valid_dataset), 1))
            valid_labels = np.array([])#np.zeros((len(valid_dataset), 1))
            print(valid_preds.shape)
            running_loss = 0.0
            for i, x_batch in enumerate(tk0):
                inputs = x_batch["image"]
                labels = x_batch["labels"].view(-1,1+4)
                weights = x_batch["weight"].view(-1, 1)
                inputs = inputs.to(device, dtype=torch.float)
                labels = labels.to(device, dtype=torch.float)
                weights = weights.to(device, dtype=torch.float)
                outputs = model(inputs)
                loss = criterion(weights,outputs, labels)
                valid_preds = np.concatenate([valid_preds,outputs[:,0].detach().cpu().squeeze().numpy().ravel()])
                valid_labels = np.concatenate([valid_labels,labels[:,0].detach().cpu().squeeze().numpy().ravel()])

                running_loss += loss.item() * inputs.size(0)
            valid_loss = running_loss / len(valid_data_loader)
            valid_preds = np.array(valid_preds/1.).flatten()
            valid_labels = np.array(valid_labels/1.).flatten()
            optR = OptimizedRounder()
            optR.fit(valid_preds, valid_labels)
            coef = optR.coefficients()
            print(coef)
            print(valid_preds.shape,valid_labels.shape)
            valid_preds1 = np.zeros_like(valid_preds)
            for i, pred in enumerate(valid_preds):
                if pred < coef[0]:
                    valid_preds1[i] = 0
                elif pred >= coef[0] and pred < coef[1]:
                    valid_preds1[i] = 1
                elif pred >= coef[1] and pred < coef[2]:
                    valid_preds1[i] = 2
                elif pred >= coef[2] and pred < coef[3]:
                    valid_preds1[i] = 3
                else:
                    valid_preds1[i] = 4
            qw_kappa1 = cohen_kappa_score(valid_labels,valid_preds1,weights='quadratic')
            # raw kappa
            valid_preds2 = valid_preds
            coef = [0.5, 1.5, 2.5, 3.5]
            valid_preds2 = np.zeros_like(valid_preds)
            for i, pred in enumerate(valid_preds):
                if pred < coef[0]:
                    valid_preds2[i] = 0
                elif pred >= coef[0] and pred < coef[1]:
                    valid_preds2[i] = 1
                elif pred >= coef[1] and pred < coef[2]:
                    valid_preds2[i] = 2
                elif pred >= coef[2] and pred < coef[3]:
                    valid_preds2[i] = 3
                else:
                    valid_preds2[i] = 4
            qw_kappa2 = cohen_kappa_score(valid_labels,valid_preds2,weights='quadratic')
            if qw_kappa1>0.8:
                torch.save(model.state_dict(), "fold{}_model_ep{}.bin".format(iiii,epoch))
            print('Training Loss: {:.4f}'.format(epoch_loss),'Valid Loss: {:.4f}'.format(valid_loss),'opt QW kapa: {:.4f}'.format(qw_kappa1),
                 'raw QW kapa: {:.4f}'.format(qw_kappa2))

        time_elapsed = time.time() - since
        print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))

        print('done')

2929 733
4.0
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
fold 0 Epoch 0/14
----------
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
(0,)
[0.50666602 1.5247565  2.51316309 3.29306619]
(733,) (733,)
Training Loss: 18.1579 Valid Loss: 13.3524 opt

In [0]:
!ls

auto_augment.py       fold1_model_ep6.bin   fold3_model_ep3.bin
fold0_model_ep0.bin   fold1_model_ep7.bin   fold3_model_ep4.bin
fold0_model_ep10.bin  fold1_model_ep8.bin   fold3_model_ep5.bin
fold0_model_ep11.bin  fold1_model_ep9.bin   fold3_model_ep6.bin
fold0_model_ep12.bin  fold2_model_ep0.bin   fold3_model_ep7.bin
fold0_model_ep13.bin  fold2_model_ep10.bin  fold3_model_ep8.bin
fold0_model_ep14.bin  fold2_model_ep11.bin  fold3_model_ep9.bin
fold0_model_ep1.bin   fold2_model_ep12.bin  fold4_model_ep0.bin
fold0_model_ep2.bin   fold2_model_ep13.bin  fold4_model_ep10.bin
fold0_model_ep3.bin   fold2_model_ep14.bin  fold4_model_ep11.bin
fold0_model_ep4.bin   fold2_model_ep1.bin   fold4_model_ep12.bin
fold0_model_ep5.bin   fold2_model_ep2.bin   fold4_model_ep13.bin
fold0_model_ep6.bin   fold2_model_ep3.bin   fold4_model_ep14.bin
fold0_model_ep7.bin   fold2_model_ep4.bin   fold4_model_ep1.bin
fold0_model_ep8.bin   fold2_model_ep5.bin   fold4_model_ep2.bin
fold0_model_ep9.bin   fold2_model_e

In [0]:
'''!git clone https://github.com/thatbrguy/Dropbox-Uploader.git
#!cd see link https://www.freecodecamp.org/news/how-to-transfer-large-files-to-google-colab-and-remote-jupyter-notebooks-26ca252892fa/ for detail
!chmod +x Dropbox-Uploader/dropbox_uploader.sh'''

'!git clone https://github.com/thatbrguy/Dropbox-Uploader.git\n#!cd see link https://www.freecodecamp.org/news/how-to-transfer-large-files-to-google-colab-and-remote-jupyter-notebooks-26ca252892fa/ for detail\n!chmod +x Dropbox-Uploader/dropbox_uploader.sh'

In [0]:
'''!bash Dropbox-Uploader/dropbox_uploader.sh #no need, just run next cell and go on'''

'!bash Dropbox-Uploader/dropbox_uploader.sh #no need, just run next cell and go on'

In [0]:
'''!echo "ahRVwuNlwQAAAAAAAAAAQr-R6312ZlGCtBbCzzdpJhQT9vncvE1r3XqudR3XpkJw" > token.txt'''

'!echo "ahRVwuNlwQAAAAAAAAAAQr-R6312ZlGCtBbCzzdpJhQT9vncvE1r3XqudR3XpkJw" > token.txt'

In [0]:
'''!bash Dropbox-Uploader/dropbox_uploader.sh'''

'!bash Dropbox-Uploader/dropbox_uploader.sh'

In [0]:
'''!bash Dropbox-Uploader/dropbox_uploader.sh upload model_ep15.bin model_ep15.bin'''

'!bash Dropbox-Uploader/dropbox_uploader.sh upload model_ep15.bin model_ep15.bin'